In [ ]:
'''
Parse the Mona Lisa image to grab all the hexadecimal values.
'''
import json
from openai import OpenAI
from PIL import Image
import textwrap

client = OpenAI()
image = Image.open('imgs/input.jpg')
pixels = image.load() 
width, height = image.size
hexadecimals = []
for x in range(width):
    for y in range(height):
        r, g, b = pixels[x, y]
        hexadecimals.append((x, y, f"#{r:02x}{g:02x}{b:02x}"))

print(len(hexadecimals))

In [ ]:
'''
Creates the list of tasks to run.
'''
tasks = []

for color in hexadecimals:
    task = {
        "custom_id": f"task-{color[0]}_{color[1]}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "temperature": 0.1,
            "response_format": {"type": "json_object"},
            "messages": [
                {"role": "system", "content": "You are a master of illustrative descriptions."},
                {
                    "role": "user",
                    "content": textwrap.dedent(f"""
            Color: {color[2]}
            ----
            Please respond in JSON, with one key - `description`. Describe the color using imagery. Don't include the hexadecimal value in the response.
            """),
                },
            ],
        },
    }

    tasks.append(task)

In [ ]:
'''
Saves it to a file which will be uploaded.
'''
file_name = "batchinput.jsonl"

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

In [ ]:
'''
Uploads the input file.
'''
batch_input_file = client.files.create(
  file=open("batchinput.jsonl", "rb"),
  purpose="batch"
)

In [ ]:
'''
Input file object returned by OpenAI
'''
batch_input_file

In [ ]:
'''
Creates a batch job using the id in the input file object.
'''
batch_input_file_id = batch_input_file.id

batch_job = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "The Mona Lisa Descriptions"
    }
)

In [ ]:
'''
Batch job object
'''
batch_job

In [ ]:
'''
List of batches (Useful if you have lost your batch job id. Can also be found using the website.)
'''
client.batches.list(limit=10)

In [ ]:
'''
Returns the batch job object, updated with the current status of the job and other important details.
Can often have an almost instantaneous turnaround. Depends on the batch size.
'''
batch_job_check = client.batches.retrieve(batch_job.id)

print(batch_job_check)
print()
print(f'{batch_job_check.status=}')
print(f'{batch_job_check.request_counts=}')


In [ ]:
'''
Saves the output.
'''
file_response = client.files.content(batch_job_check.output_file_id)
result_file_name = "batchoutput.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(file_response.content)